### Youtube Sentence With Word And Order With Timestamp

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
word_num = 300
df_word = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/Turkish/Word Tokenize Merge/Youtube_Word_Tokenize_Merge.csv") # word merge dataframe
df_word = df_word.head(word_num)
df_word

,word,frequency
0,bir,1173600
1,bu,840135
2,ve,590780
3,çok,497301
4,da,486717
...,...,...
295,hasta,15873
296,kötü,15853
297,birazcık,15834
298,geldim,15826


In [4]:
data_kind = "sentence"
df_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/Turkish/Sentence Clean Merge/Clean_Youtube_{data_kind.lower().capitalize()}_Merge_Result.csv")
df_sent

,start_time,end_time,sentence,video_id
0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM
1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM
2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM
3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM
4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM
...,...,...,...,...
387903,00:02:11.720,00:02:14.440,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE
387904,00:02:15.560,00:02:19.320,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE
387905,00:02:20.320,00:02:21.200,sahi,7R3K6EDaBGE
387906,00:02:21.720,00:02:24.480,diğer eşi sende mi hala,7R3K6EDaBGE


In [5]:
word_list = df_word.iloc[:,0].values.tolist()

In [6]:
sent_list  = df_sent.iloc[:,2].values.tolist()

In [7]:
resultlist = []

In [8]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(sent_list):
    mergelist = []
    try:
        word = sent_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(sent_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, sent_list) # string_word liste

In [9]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result.drop_duplicates(inplace=True)
df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
df_sent_merge = pd.merge(df_result, df_sent, how="inner", on=f"{data_kind.lower()}")
df_sent_merge.drop_duplicates(inplace=True)
df_sent_with_words = df_sent_merge.sort_values(by=["video_id","start_time"], ascending=True)
df_sent_with_words.reset_index(drop=True, inplace=True)
df_sent_with_words

,sentence,start_time,end_time,video_id
0,yeni,00:00:02.304,00:00:02.816,0Bs9I3H1OJk
1,ne kadar,00:00:30.976,00:00:31.488,0Bs9I3H1OJk
2,bunun için,00:01:07.072,00:01:07.584,0Bs9I3H1OJk
3,şu anda,00:02:15.168,00:02:15.680,0Bs9I3H1OJk
4,birkaç gün içerisinde,00:02:34.112,00:02:34.624,0Bs9I3H1OJk
...,...,...,...,...
16550,çok güzel olmuş,00:02:33.583,00:02:35.008,zwWS6yifThY
16551,var,00:08:13.490,00:08:19.860,zzXTIIo7EvM
16552,ben,00:12:56.180,00:13:04.974,zzXTIIo7EvM
16553,burası da,00:22:55.450,00:23:01.260,zzXTIIo7EvM


In [10]:
#df_merge_result.to_csv(f"Youtube_{data_kind.capitalize()}_With_{word_num}_Words.csv", index=False)

In [11]:
df_sent_with_words["with_word"] = 1
df_sent_with_words

,sentence,start_time,end_time,video_id,with_word
0,yeni,00:00:02.304,00:00:02.816,0Bs9I3H1OJk,1
1,ne kadar,00:00:30.976,00:00:31.488,0Bs9I3H1OJk,1
2,bunun için,00:01:07.072,00:01:07.584,0Bs9I3H1OJk,1
3,şu anda,00:02:15.168,00:02:15.680,0Bs9I3H1OJk,1
4,birkaç gün içerisinde,00:02:34.112,00:02:34.624,0Bs9I3H1OJk,1
...,...,...,...,...,...
16550,çok güzel olmuş,00:02:33.583,00:02:35.008,zwWS6yifThY,1
16551,var,00:08:13.490,00:08:19.860,zzXTIIo7EvM,1
16552,ben,00:12:56.180,00:13:04.974,zzXTIIo7EvM,1
16553,burası da,00:22:55.450,00:23:01.260,zzXTIIo7EvM,1


In [12]:
df_with_word = pd.merge(df_sent, df_sent_with_words, how="left", on=["sentence","start_time","end_time","video_id"])
df_with_word.fillna(0, inplace=True)
df_with_word

,start_time,end_time,sentence,video_id,with_word
0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM,0.0
1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM,0.0
2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM,0.0
3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM,0.0
4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM,0.0
...,...,...,...,...,...
387903,00:02:11.720,00:02:14.440,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE,0.0
387904,00:02:15.560,00:02:19.320,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE,0.0
387905,00:02:20.320,00:02:21.200,sahi,7R3K6EDaBGE,0.0
387906,00:02:21.720,00:02:24.480,diğer eşi sende mi hala,7R3K6EDaBGE,0.0


In [13]:
df_with_word.with_word.value_counts()

0.0    371353
1.0     16555
Name: with_word, dtype: int64

In [ ]:
# First Code Running For All Sentence
# zeros = []
# ones = []
# zero_list = []
# one_list = []
# for i in range(len(df_with_word)):    
#     if df_with_word.loc[i,"with_word"] == 0:
#         zeros.append(i)
#         try:
#             one_list.append((ones[0],len(ones)))
#         except:
#             pass
#         ones = []
#     else:
#         ones.append(i)
#         try:
#             zero_list.append((zeros[0],len(zeros)))
#         except:
#             pass
#         zeros = []
# try:
#     one_list.append((ones[0],len(ones)))
# except:
#     pass
# try:
#     zero_list.append((zeros[0],len(zeros)))
# except:
#     pass# 

In [15]:
# Try Code (will test)
zeros = []
ones = []
zero_list = []
one_list = []
for i in range(len(df_with_word)):
    videoid = df_with_word.loc[i,"video_id"]    
    if df_with_word.loc[i,"with_word"] == 0:
        zeros.append(i)
        try:
            one_list.append((ones[0],len(ones)))
        except:
            pass
        ones = []
    else:
        ones.append(i)
        try:
            zero_list.append((zeros[0],len(zeros)))
        except:
            pass
        zeros = []

    try:
        if df_with_word.loc[i+1,"video_id"] != videoid:
            try:
                one_list.append((ones[0],len(ones)))
                ones = []
            except:
                pass
            try:
                zero_list.append((zeros[0],len(zeros)))
                zeros = []
            except:
                pass
        else:
            pass
    except:
        pass

try:
    one_list.append((ones[0],len(ones)))
except:
    pass
try:
    zero_list.append((zeros[0],len(zeros)))
except:
    pass

In [16]:
len(one_list)

13996

In [17]:
len(zero_list)

18784

In [18]:
df_one = pd.DataFrame(one_list)
df_one.rename(columns={0:"index",1:"length"}, inplace=True)
df_one.drop_duplicates(inplace=True)
df_one

,index,length
0,59,1
1,65,1
2,81,1
3,153,1
4,162,1
...,...,...
13991,387845,1
13992,387851,1
13993,387853,1
13994,387866,1


In [23]:
df_one["length"].sum()

16555

In [24]:
df_one.sort_values(by="length", ascending=False)

,index,length
10544,286907,39
2157,39290,31
9394,256862,29
6287,145435,22
9415,257157,17
...,...,...
5043,109976,1
5044,109983,1
5045,110066,1
5046,110236,1


In [20]:
df_zero = pd.DataFrame(zero_list)
df_zero.rename(columns={0:"index",1:"length"}, inplace=True)
df_zero.drop_duplicates(inplace=True)
df_zero

,index,length
0,0,30
1,30,22
2,52,7
3,60,5
4,66,15
...,...,...
18779,387846,5
18780,387852,1
18781,387854,12
18782,387867,20


In [22]:
df_zero["length"].sum()

371353

In [ ]:
df_word_result = pd.DataFrame()
for i in word_list:
    word_in_video = df_sent_with_words[df_sent_with_words.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] # try and except will try for sample 
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result = df_word_result.sort_values(by=["video_id","start_time"], ascending=True)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

In [ ]:
#df_word_result.to_csv(f"Youtube_{data_kind.capitalize()}_With_{word_num}_Word_And_Words.csv", index=False)

In [ ]:
df_group = pd.DataFrame(df_word_result.groupby("video_id")["word"].nunique())
df_group.reset_index(inplace=True)
df_group.rename(columns={"word":"frequency"}, inplace=True)
df_group.sort_values(by="frequency", ascending=False, inplace=True)
df_group.reset_index(drop=True, inplace=True)
df_group 

In [ ]:
df_word_result[df_word_result.video_id == "UDDfBEUGY60"]